## This notebook aims to verify the use of standing-laying classifer and find the threshold value for result generation 

### About standing laying classifier Model(Validation accuracy is 99%)
1. Give result between 0 and 1
2. Less the 1 but close to 1  means children is standing 
3. Greater then 0 but close to 0 means children is laying
4. Any random value between 0.1 and 0.9 - some body parts not present in image

In [ ]:
import glob2 as glob
import numpy as np
import cv2
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from utils import (REPO_DIR,get_timestamp_from_pcd,get_timestamp_from_rgb,find_closest,standing_laying_predict,download_model)
from azureml.core import Workspace

#### Mount the dataset with compute instance or downlaod the dataset locally 
#### Dataset use are :- anon-rgb-classification

In [ ]:
DATA=REPO_DIR / 'data' / 'anon-rgb-classification' / 'test' / 'scan'
qr_codes = glob.glob(os.path.join(DATA, "qrcode/*")) # Find all the qr code present in dataset

### Find the timestamp of PCDs and RGBs artifacts. And find the closest timestamp of RGBs with every PCDs in this way we can say the both artifacts is at the same time.

In [ ]:
qrcode_pcd_rgb=[]
for qr_code in qr_codes:
    pcd_paths = glob.glob(os.path.join(qr_code, "measure/*/pc/*"))   # Find all the pcd in the particular qrcode
    rgb_paths = glob.glob(os.path.join(qr_code, 'measure/*/rgb/*'))  # Find all the rgb in the particular qrcode
    pcd_timestamp=[]
    for pcd_path in pcd_paths:
        time_stamp=get_timestamp_from_pcd(pcd_path)
        pcd_timestamp.append(time_stamp)
    rgb_timestamp=[]
    for rgb_path in rgb_paths:
        rgb_timestamp.append(get_timestamp_from_rgb(rgb_path))
    rgb_timestamp=np.asarray(rgb_timestamp)
    i=0
    files=[]
    for pcd_s in pcd_timestamp:
        nn = find_closest(rgb_timestamp, pcd_s)
        files.append([rgb_paths[nn],pcd_paths[i]])
        i=i+1
    qrcode_pcd_rgb.append(files)

### Download Standing-laying classifer model

In [ ]:
ws = Workspace.from_config()
EXPERIMENT_NAME='q4-rgb-plaincnn-classifaction-standing-lying-8k'
RUN_ID='q4-rgb-plaincnn-classifaction-standing-lying-8k_1602316038_3ebdb326'
INPUT_LOCATION = 'outputs/best_model.h5'
OUTPUT_LOCATION= REPO_DIR / 'src' / 'analyses' / 'reliability' / 'Standing-laying'
download_model(ws, EXPERIMENT_NAME, RUN_ID, INPUT_LOCATION, OUTPUT_LOCATION)

#### Find the prediction of standing-laying classifier on all corresponding rgb images

In [ ]:
model = load_model(OUTPUT_LOCATION / 'best_model.h5') 
qrcodes_predicts=standing_laying_predict(qrcode_pcd_rgb,model)

## Visualize

#### Image of standing child with 0.32 score

In [ ]:
%pylab inline
img = mpimg.imread(str(qrcodes_predicts[7][16][2]))
imgplot = plt.imshow(img)
plt.show()
print(qrcodes_predicts[7][16][0])

#### Image of standing child with 0.99 score

In [ ]:
%pylab inline
img = mpimg.imread(str(qrcodes_predicts[7][0][2]))
imgplot = plt.imshow(img)
plt.show()
print(qrcodes_predicts[7][0][0])

### Findings:
1. Threshold can be 0.98 for standing and 0.02 for laying
2. If for standing confidence score is below 0.98 then the corresponding PCDs is discarted
3. If the laying confidece score is greater then 0.02 then the corresponding PCDs is discarted

### Upcoming Task:
1. Create unclean dataset for proper evaluation 
2. Filter out inaccurate data
3. Check the ML performance 
4. Evaluation of model on based of standing/laying
5. Add more class to the classifier